# RNA cis-effects with t tests for 8p

Here we look for genes IN the event discussed that are affected by the arm-level event. We will find these effects by performing a series of t-tests comparing the transcriptomics values of the patients with the event against those without the event.

## Setup (Install necessary packages)

We will start by importing necessary packages and collecting all of the transcriptomics data we will need to run the tests. The cancer types analyzed should have been determined in 01_event_basic_info where we determine which types of cancer seem to have the event we are looking at.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import cptac
from scipy import stats
import cnvutils
import cptac.utils

In [2]:
# These variables specify which chromosome and arm we're working on, and whether to do cis or trans
CHROMOSOME = '8'
ARM = 'p'
CIS_OR_TRANS = "cis"

In [3]:
if ARM == "p":
    EVENT_COLUMN = "loss_event"
    EXCLUDE_COLUMN = "gain_event"
    EVENT_START = 0
    EVENT_END = 30794385
    
elif ARM == "q":
    EVENT_COLUMN = "gain_event"
    EXCLUDE_COLUMN = "loss_event"
    EVENT_START = 80794385
    EVENT_END = 130794385

else:
    raise ValueError("Invalid value for ARM variable.")

In [4]:
cancer_types = {
    "brca": cptac.Brca,
    "colon": cptac.Colon,
    "hnscc": cptac.Hnscc,
    "lscc": cptac.Lscc,
    "luad": cptac.Luad,
    "ovarian": cptac.Ovarian
}

## Select the RNAs we're interested in

If we're looking at cis effects, we select RNAs within the event. If we're looking at trans effects, we select RNAs outside of the event.

In [5]:
selected_genes = cnvutils.\
get_event_genes(
    chrm=CHROMOSOME,
    event_start=EVENT_START,
    event_end=EVENT_END,
    cis_or_trans=CIS_OR_TRANS
)["Name"].\
drop_duplicates(keep="first")

def load_transcriptomics_tumor(dataset_func):
    return dataset_func().get_transcriptomics(tissue_type="tumor")

transcriptomics = {}

for cancer_type in cancer_types.keys():
    df = load_transcriptomics_tumor(cancer_types[cancer_type]).transpose()

    if df.index.nlevels == 1:
        df = df[df.index.isin(selected_genes)]
    else:
        df = df[df.index.isin(selected_genes, level="Name")]

    transcriptomics[cancer_type] = df

cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (/home/caleb/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py, line 11)


## Append Event Data

We now append the data from the event table that should have been created in a previous notebook.

In [6]:
has_event = dict()
for cancer_type in transcriptomics.keys():
    df = transcriptomics[cancer_type]
    df = df.transpose()
    event = pd.read_csv(f'{cancer_type}_has_event.tsv', sep='\t', index_col=0)
    if EXCLUDE_COLUMN:
        event.drop(EXCLUDE_COLUMN, axis=1, inplace=True)
    event.index.rename('Name')
    df = df.join(event)
    df = df.dropna(subset=[EVENT_COLUMN])
    has_event[cancer_type] = df[EVENT_COLUMN]
    transcriptomics[cancer_type] = df

## Run T-Tests

In [7]:
results_df = None
for cancer_type in transcriptomics.keys():
    prot_df = transcriptomics[cancer_type]
    results = cptac.utils.wrap_ttest(
        df=prot_df, 
        label_column=EVENT_COLUMN,
        correction_method="fdr_bh",
        return_all=True,
        quiet=True
    )   
    results.set_index('Comparison', inplace=True)
    if isinstance(results.index[0], tuple):
        results[['Name', f'{cancer_type}_Database_ID']] = pd.DataFrame(
            results.index.values.tolist(),
            index=results.index
        )
        results.set_index(['Name', f'{cancer_type}_Database_ID'], inplace=True)
    else:
        results.index.name='Name'
    results.rename(columns={'P_Value': f'{cancer_type}_pvalue'}, inplace=True)
    if results_df is None:
        results_df = results
    else:
        results_df = results_df.join(results)

## Append Difference Data

We want to collect one more piece of information about the data. We want to find the difference between the averages of the two groups. This will tell us if there is a positive or negative coorelation between the event and the proteomic data and can give us some indication of the strength of the coorelation. 

In [8]:
def get_diff(col, event):
    has_event = col[event]
    invert_list = [not x for x in event]
    no_event = col[invert_list]
    event_avg = has_event.mean()
    no_event_avg = no_event.mean()
    return event_avg - no_event_avg

In [9]:
diff_df = None
for cancer_type in transcriptomics.keys():
    df = transcriptomics[cancer_type]
    df = df.drop(EVENT_COLUMN, axis=1)
    results = df.apply(lambda x: get_diff(x, has_event[cancer_type]))
    df = pd.DataFrame(results)
    if isinstance(df.index[0], tuple):
        df[['Name', f'{cancer_type}_Database_ID']] = pd.DataFrame(df.index.values.tolist(), index=df.index)
        df.set_index(['Name', f'{cancer_type}_Database_ID'], inplace=True)
    else:
        df.index.name='Name'
    df.rename(columns={0: f'{cancer_type}_diff'}, inplace=True)
    if diff_df is None:
        diff_df = df
    else:
        diff_df = diff_df.join(df)

## Join the tables and save

We now join the difference table and the results table together. We also save the table to a tsv for use in future analyses. 

In [10]:
results_df = results_df.join(diff_df)

In [11]:
results_df

,brca_pvalue,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue,brca_diff,colon_diff,hnscc_diff,lscc_diff,luad_diff,ovarian_diff
Name,,,,,,,,,,,,
CNOT7,3.914365e-10,1.435448e-10,1.665203e-09,3.304521e-05,3.046598e-09,3.514387e-04,-0.976367,-0.752688,-0.608418,-0.379001,-0.527819,-15.138692
INTS10,3.914365e-10,5.060918e-15,1.217401e-08,2.930066e-06,6.084956e-12,2.980770e-05,-1.081133,-0.890928,-0.553038,-0.439746,-0.569810,-10.005264
MCPH1,4.035433e-10,1.088141e-11,7.875888e-08,2.507263e-11,3.322089e-10,3.477368e-05,-0.949365,-0.742764,-0.446840,-0.577884,-0.564321,-2.963324
ELP3,2.250854e-09,1.061800e-18,2.460093e-09,2.929160e-06,1.442990e-07,6.150840e-05,-0.991932,-0.794019,-0.519073,-0.365279,-0.484852,-7.420865
CHMP7,3.207350e-09,4.153119e-13,7.296693e-08,1.362912e-13,1.076006e-08,NaN,-0.904405,-0.640393,-0.477455,-0.575689,-0.529063,NaN
CCDC25,5.389864e-09,5.022518e-10,1.295629e-08,8.880786e-04,4.178903e-09,2.972695e-04,-1.146690,-0.854391,-0.593974,-0.287809,-0.531863,-5.514267
CCAR2,7.827383e-09,2.673128e-06,2.359004e-06,2.012667e-05,7.586921e-14,2.248690e-02,-0.943937,-0.536681,-0.435945,-0.332146,-0.604200,-4.107823
ZNF395,1.092787e-07,5.366167e-05,1.135782e-02,1.421384e-01,3.718213e-05,7.280114e-02,-1.151203,-0.409066,-0.463445,-0.201753,-0.505567,-2.637418
PPP2R2A,1.092787e-07,1.651572e-13,7.965704e-07,2.000672e-08,2.149935e-08,2.437569e-06,-1.000275,-0.786856,-0.501980,-0.473264,-0.596554,-7.396398


In [12]:
# This will save the resulting table in the same directory as this notebook.
# Modify if you would like to save to a different location.
results_df.to_csv(f"{CHROMOSOME}{ARM}_{CIS_OR_TRANS}RNAeffects_ttest.tsv", sep='\t')